# Análisis del Pipeline de Entrenamiento del Modelo

**Autor:** Julio Clavijo
**Fecha:** 23/10/2025

## Objetivo
Este notebook documenta y ejecuta el pipeline de entrenamiento del modelo de Machine Learning para la predicción de la demanda energética en Barcelona. El proceso completo, desde la carga de datos hasta la interpretación del modelo, se detalla a continuación.

In [ ]:
# --- 0. IMPORTACIÓN DE LIBRERÍAS ---
# Se importan todas las herramientas necesarias para el análisis.
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error
import shap
import matplotlib.pyplot as plt
import os
from google.oauth2 import service_account
from dotenv import load_dotenv

print("Librerías importadas con éxito.")

## Paso 1: Carga de Datos desde BigQuery

Nos conectamos a nuestro Data Warehouse en BigQuery para cargar la tabla `gold_data.modelo_final`. Esta tabla es el resultado de todo el pipeline de ETL y contiene los datos limpios, unidos y listos para el modelado.

In [ ]:
# --- 1. CARGA DE DATOS DESDE BIGQUERY ---
print("--- Iniciando el pipeline de entrenamiento de modelo ---")
print("Paso 1: Cargando datos desde la tabla Gold 'modelo_final'...")

# Carga las variables de entorno para la autenticación
load_dotenv()
GCP_KEY_PATH = os.getenv("GCP_SERVICE_ACCOUNT_KEY_PATH")
PROJECT_ID = "datamanagementbi"
TABLE_ID = "gold_data.modelo_final"

# Autenticación y ejecución de la consulta
try:
    credentials = service_account.Credentials.from_service_account_file(GCP_KEY_PATH)
    query = f"SELECT * FROM `{PROJECT_ID}.{TABLE_ID}` ORDER BY fecha, id_tramo_horario"
    df = pd.read_gbq(query, project_id=PROJECT_ID, credentials=credentials, progress_bar_type=None)
    print(f"Carga de datos completada. Se han cargado {len(df)} registros.")
    display(df.head()) # Mostramos las primeras 5 filas para verificar
except Exception as e:
    print(f"Error al cargar datos desde BigQuery: {e}")

## Paso 2: Preparación de Datos y Feature Engineering

Realizamos los últimos ajustes al DataFrame:
1.  Aseguramos que la columna `fecha` sea de tipo `datetime` y la establecemos como el índice de la tabla. Esto es fundamental para trabajar con series temporales.
2.  Convertimos las columnas de IDs y nombres categóricos al tipo `category` de Pandas. Esto optimiza el uso de memoria y es la forma correcta de indicar a librerías como XGBoost que estas columnas son etiquetas, no valores numéricos.

In [ ]:
# --- 2. PREPARACIÓN DE DATOS Y FEATURE ENGINEERING ---
print("\nPaso 2: Preparando datos para el entrenamiento...")
df['fecha'] = pd.to_datetime(df['fecha'])
df.set_index('fecha', inplace=True)

categorical_cols = [
    'id_geografia', 'id_tramo_horario', 'id_sector_economico', 
    'dia_de_la_semana_nombre', 'nombre_barrio', 'nombre_distrito'
]
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')
print("Tipos de datos preparados y columnas categóricas configuradas.")
df.info()

## Paso 3 y 4: Selección de Features y División de Datos

Separamos nuestro conjunto de datos en:
- **Target (`y`):** La variable que queremos predecir (`consumo_kwh`).
- **Features (`X`):** Todas las "pistas" que usará el modelo para hacer la predicción.

Luego, dividimos los datos de forma **cronológica**. Usamos el 80% más antiguo para entrenar y reservamos el 20% más reciente para evaluar el modelo, simulando un escenario de predicción real.

In [ ]:
# --- 3. SELECCIÓN DE CARACTERÍSTICAS Y OBJETIVO ---
target = 'consumo_kwh'
features = [col for col in df.columns if col not in [
    target, 'nombre_municipio', 'festivo_descripcion'
]]
X = df[features]
y = df[target]

# --- 4. DIVISIÓN DE DATOS (TRAIN/TEST SPLIT PARA SERIES TEMPORALES) ---
test_size = int(len(df) * 0.2)
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]
print(f"Datos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")

## Paso 5 y 6: Entrenamiento y Evaluación del Modelo

Utilizamos **XGBoost**, un potente algoritmo de Gradient Boosting, para entrenar nuestro modelo de regresión.
- **Entrenamiento:** El modelo aprende los patrones a partir del conjunto `_train`. Usamos `early_stopping_rounds` para evitar el sobreajuste y optimizar el tiempo de entrenamiento.
- **Evaluación:** Medimos el rendimiento del modelo sobre el conjunto `_test` (datos que nunca ha visto) usando la métrica **MAPE** (Mean Absolute Percentage Error).

In [ ]:
# --- 5. ENTRENAMIENTO DEL MODELO XGBOOST ---
print("\nPaso 5: Entrenando el modelo XGBoost...")
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    early_stopping_rounds=10,
    eval_metric='mape',
    enable_categorical=True
)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
print("Modelo entrenado con éxito.")

# --- 6. EVALUACIÓN DEL MODELO ---
print("\nPaso 6: Evaluando el rendimiento del modelo...")
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("="*50)
print(f"  RESULTADO FINAL -> MAPE: {mape:.4f}")
print("="*50)

## Paso 7: Interpretación del Modelo (SHAP)

Finalmente, utilizamos la librería **SHAP (SHapley Additive exPlanations)** para entender qué características han sido más importantes para las predicciones del modelo. Esto nos permite "abrir la caja negra" y explicar por qué el modelo toma ciertas decisiones.

El siguiente gráfico muestra la importancia media de cada característica. Una barra más larga indica un mayor impacto en la predicción del consumo.

In [ ]:
# --- 7. INTERPRETACIÓN DEL MODELO (SHAP) ---
print("\nPaso 7: Generando gráfico de importancia de características (SHAP)...")

explainer = shap.Explainer(model)
shap_values = explainer(X_test, check_additivity=False)

# Generamos y mostramos el gráfico
fig, ax = plt.subplots(figsize=(10, 8))
shap.summary_plot(shap_values, X_test, show=False, plot_type="bar", ax=ax)
fig.tight_layout()
plt.show()

# Guardamos la figura
output_path = os.path.join(os.getcwd(), 'shap_summary_final.png')
fig.savefig(output_path)
plt.close(fig)
print(f"Gráfico guardado con éxito en: {output_path}")